## Replacing invalid string values

Some columns contain incorrect placeholders for missing data, such as `"error"`, `"unknown"`, or `"nan"`.  
These should be consistently replaced with proper `NaN` values for accurate analysis and type conversion.



In [ ]:
import pandas as pd 
import numpy as np

# Load dataset
df = pd.read_csv("../data/raw/dirty_cafe_sales.csv")


# Create function to replace the incorrect missing indicators
def clean_incorrect_missing_indicators(columns: list[str], df: pd.DataFrame = df):
    """
        Replaces string-based placeholders for missing values (e.g., "error", "unknown", "nan") 
        with actual NumPy NaN values in the specified columns.

        Parameters:
            columns (list[str]): List of column names to apply the cleaning on.
            df (pd.DataFrame): The input DataFrame (default is `df`).

        Returns:
            pd.DataFrame: A new DataFrame with cleaned missing value indicators.
    """

    df_copy = df.copy()
    string_to_replace = ["error", "nan", "unknown"]
    for col in columns:
        if col in df.columns:
            def clean_value(x):
                if isinstance(x, str):
                    lower_x = x.strip().lower()
                    if lower_x in string_to_replace:
                        return np.nan
                return x
            df_copy[col] = df_copy[col].apply(clean_value)
    return df_copy    

# Call the function
df = clean_incorrect_missing_indicators(columns=df.columns.to_list(), df=df)

In [131]:
df["Location"].unique()

array(['Takeaway', 'In-store', nan], dtype=object)

## Convert feature data types

We will convert numerical and date columns to appropriate types using `pd.to_numeric` and `pd.to_datetime`.


In [132]:
# Lets turn numerical features to int
for col in ['Price Per Unit', 'Total Spent', 'Quantity']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    10000 non-null  object        
 1   Item              9031 non-null   object        
 2   Quantity          9521 non-null   float64       
 3   Price Per Unit    9467 non-null   float64       
 4   Total Spent       9498 non-null   float64       
 5   Payment Method    6822 non-null   object        
 6   Location          6039 non-null   object        
 7   Transaction Date  9540 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 625.1+ KB


## Handle missing values

In [133]:
# Check the missing values
(df.isna().sum() / len(df) * 100).sort_values(ascending=False)

Location            39.61
Payment Method      31.78
Item                 9.69
Price Per Unit       5.33
Total Spent          5.02
Quantity             4.79
Transaction Date     4.60
Transaction ID       0.00
dtype: float64

#### The first two features have more than 30% missing values. We need to impute them.  
#### Let's check the distribution of available values to decide how to fill the missing entries.


In [134]:
# Lets check the ration of the avaliable values 
for col in [
       'Payment Method', 'Location', 'Item']:
    print(df[col].value_counts(normalize=True))

Payment Method
Digital Wallet    0.335825
Credit Card       0.333187
Cash              0.330988
Name: proportion, dtype: float64
Location
Takeaway    0.500414
In-store    0.499586
Name: proportion, dtype: float64
Item
Juice       0.129664
Coffee      0.129000
Salad       0.127118
Cake        0.126121
Sandwich    0.125235
Smoothie    0.121360
Cookie      0.120917
Tea         0.120585
Name: proportion, dtype: float64


####  Based on the ration of the 'Payment Method', 'Item' and 'Location' we will fill the missing values by proportion


In [135]:
def proportional_fill(df: pd.DataFrame, column: str):
    """
        Fill nan values of the column based on the avaliable valuables ratio

        Props:
            df: pd.DataFrame
            column: str - name of thh dataframe feature

        Return: 
            df: pd.DataFrame - with the filled missing values
    """
    propbs = df[column].value_counts(normalize=True)
    n_missing = df[column].isna().sum()

    input_values = np.random.choice(propbs.index, size=n_missing, p=propbs.values)
    df.loc[df[column].isna(), column] = input_values
    return df

#### Fill out missing values
* Object types will be filled based on there avaliable values proportion
* Numerical values by there median
* DateTime features will be droped as there is not data based we know when it happaned

In [136]:
df = proportional_fill(df, 'Location')
df = proportional_fill(df, 'Payment Method')
df = proportional_fill(df, 'Item')

# Numerical values will be filled out with there median
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Price Per Unit'].median()) 
df['Total Spent'] = df['Total Spent'].fillna(df['Total Spent'].median()) 
df['Quantity'] = df['Quantity'].fillna(df['Quantity'].median()) 

# Drop DateTime
df = df.dropna(subset=["Transaction Date"])

In [137]:
# Lets check if is there any missing values
(df.isna().sum() / len(df) * 100).sort_values(ascending=False)

Transaction ID      0.0
Item                0.0
Quantity            0.0
Price Per Unit      0.0
Total Spent         0.0
Payment Method      0.0
Location            0.0
Transaction Date    0.0
dtype: float64